In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.express as px
plt.style.use('fivethirtyeight')

from sklearn.preprocessing import MinMaxScaler 

import tensorflow as tf

import numpy as np
import pandas as pd
import math

from sklearn.metrics import mean_squared_error as mse
from keras.losses import MeanAbsolutePercentageError

from keras.initializers import RandomUniform

# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')

In [2]:
df = pd.read_csv('../data/data_ambrox.csv')

df.head()

,Month,Use_Ambroxol
0,2015-01-01,3342
1,2015-02-01,3823
2,2015-03-01,3456
3,2015-04-01,4129
4,2015-05-01,2998


In [3]:
df['Month'] = pd.to_datetime(df['Month']).dt.date
df.head()

,Month,Use_Ambroxol
0,2015-01-01,3342
1,2015-02-01,3823
2,2015-03-01,3456
3,2015-04-01,4129
4,2015-05-01,2998


In [4]:
df['Month'] = pd.to_datetime(df['Month']).dt.to_period('M')
df.head()

,Month,Use_Ambroxol
0,2015-01,3342
1,2015-02,3823
2,2015-03,3456
3,2015-04,4129
4,2015-05,2998


In [5]:
df.set_index('Month', inplace=True)
df.head()

,Use_Ambroxol
Month,
2015-01,3342
2015-02,3823
2015-03,3456
2015-04,4129
2015-05,2998


In [6]:
df = df.values

scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)
df

array([[0.35735964],
       [0.72230653],
       [0.44385432],
       [0.95447648],
       [0.09635812],
       [0.4400607 ],
       [0.2754173 ],
       [0.89908953],
       [0.01517451],
       [0.44461305],
       [0.4400607 ],
       [0.29210926],
       [0.41047041],
       [0.52807284],
       [0.40515933],
       [0.35660091],
       [0.87405159],
       [0.46054628],
       [0.79666161],
       [0.26555387],
       [0.30576631],
       [0.93930197],
       [0.20789074],
       [0.        ],
       [0.60622155],
       [0.4339909 ],
       [0.09484067],
       [0.25796662],
       [0.12974203],
       [0.42033384],
       [0.52427921],
       [0.24127466],
       [0.19195751],
       [0.51138088],
       [0.10698027],
       [0.89377845],
       [0.08345979],
       [0.36798179],
       [0.26403642],
       [0.4400607 ],
       [0.50151745],
       [0.4522003 ],
       [0.32397572],
       [0.86570561],
       [0.4400607 ],
       [0.48634294],
       [0.43019727],
       [0.412

In [7]:
train_size = int(len(df) * 0.8)
test_size = (len(df)) - train_size
print(train_size, test_size)

train , test = df[0:train_size,:] , df[train_size:len(df),:]

67 17


In [8]:
def to_sequences(df, seq_size):
    x = []
    y = []

    for i in range(len(df)-seq_size):
        window = df[i:(i+seq_size), 0]
        x.append(window)
        y.append(df[i+seq_size, 0])

    return np.array(x), np.array(y)

seq_size = 5

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)

print("Shape of training set: {}".format(trainX.shape))
print("Shape of testinf set: {}".format(testX.shape))

Shape of training set: (62, 5)
Shape of testinf set: (12, 5)


In [9]:
mape = MeanAbsolutePercentageError()

### Model 1 - 2

In [10]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, input_dim=seq_size,
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(10, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(1, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1))
])

model1.compile(loss='mse',
               optimizer=tf.keras.optimizers.Adam(0.01),
               metrics=[tf.keras.metrics.MeanAbsoluteError()])

model1.fit(trainX, trainY, validation_data=(
    testX, testY), verbose=0, epochs=50, batch_size=6)

model1.evaluate(testX, testY)

trainPredict1 = model1.predict(trainX)
testPredict1 = model1.predict(testX)

trainPredict1 = scaler.inverse_transform(trainPredict1)
trainY_inverse = scaler.inverse_transform([trainY])
testPredict1 = scaler.inverse_transform(testPredict1)
testY_inverse = scaler.inverse_transform([testY])

print('Model 1')

trainScore1 = math.sqrt(mse(trainY_inverse[0], trainPredict1[:,0]))
print('Train score: %.2f' % (trainScore1))

testScore1 = math.sqrt(mse(testY_inverse[0], testPredict1[:,0]))
print('Test score: %.2f' % (testScore1))

mapeTrain1 = mape(trainY_inverse, trainPredict1)
print('MAPE Train: ', mapeTrain1.numpy())
print('Akurasi Train: ', 100 - (mapeTrain1.numpy()))

mape1 = mape(testY_inverse, testPredict1)
print('MAPE Test: ', mape1.numpy())
print('Akurasi Test: ', 100 - (mape1.numpy()))

1/1 [==============================] - 0s 26ms/step
Model 1
Train score: 274.29
Test score: 201.86
MAPE Train:  7.522051
Akurasi Train:  92.47794914245605
MAPE Test:  4.8831367
Akurasi Test:  95.11686325073242


### Model 1 - 3

In [12]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, input_dim=seq_size,
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(10, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(1, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1))
])

model1.compile(loss='mse',
               optimizer=tf.keras.optimizers.Adam(0.01),
               metrics=[tf.keras.metrics.MeanAbsoluteError()])

model1.fit(trainX, trainY, validation_data=(
    testX, testY), verbose=0, epochs=50, batch_size=6)

model1.evaluate(testX, testY)

trainPredict1 = model1.predict(trainX)
testPredict1 = model1.predict(testX)

trainPredict1 = scaler.inverse_transform(trainPredict1)
trainY_inverse = scaler.inverse_transform([trainY])
testPredict1 = scaler.inverse_transform(testPredict1)
testY_inverse = scaler.inverse_transform([testY])

print('Model 1')

trainScore1 = math.sqrt(mse(trainY_inverse[0], trainPredict1[:,0]))
print('Train score: %.2f' % (trainScore1))

testScore1 = math.sqrt(mse(testY_inverse[0], testPredict1[:,0]))
print('Test score: %.2f' % (testScore1))

mapeTrain1 = mape(trainY_inverse, trainPredict1)
print('MAPE Train: ', mapeTrain1.numpy())
print('Akurasi Train: ', 100 - (mapeTrain1.numpy()))

mape1 = mape(testY_inverse, testPredict1)
print('MAPE Test: ', mape1.numpy())
print('Akurasi Test: ', 100 - (mape1.numpy()))

1/1 [==============================] - 0s 24ms/step
Model 1
Train score: 291.33
Test score: 192.30
MAPE Train:  7.263714
Akurasi Train:  92.73628616333008
MAPE Test:  4.404525
Akurasi Test:  95.59547519683838


### Model 1 - 4

In [13]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, input_dim=seq_size,
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(10, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(1, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1))
])

model1.compile(loss='mse',
               optimizer=tf.keras.optimizers.Adam(0.01),
               metrics=[tf.keras.metrics.MeanAbsoluteError()])

model1.fit(trainX, trainY, validation_data=(
    testX, testY), verbose=0, epochs=50, batch_size=6)

model1.evaluate(testX, testY)

trainPredict1 = model1.predict(trainX)
testPredict1 = model1.predict(testX)

trainPredict1 = scaler.inverse_transform(trainPredict1)
trainY_inverse = scaler.inverse_transform([trainY])
testPredict1 = scaler.inverse_transform(testPredict1)
testY_inverse = scaler.inverse_transform([testY])

print('Model 1')

trainScore1 = math.sqrt(mse(trainY_inverse[0], trainPredict1[:,0]))
print('Train score: %.2f' % (trainScore1))

testScore1 = math.sqrt(mse(testY_inverse[0], testPredict1[:,0]))
print('Test score: %.2f' % (testScore1))

mapeTrain1 = mape(trainY_inverse, trainPredict1)
print('MAPE Train: ', mapeTrain1.numpy())
print('Akurasi Train: ', 100 - (mapeTrain1.numpy()))

mape1 = mape(testY_inverse, testPredict1)
print('MAPE Test: ', mape1.numpy())
print('Akurasi Test: ', 100 - (mape1.numpy()))

1/1 [==============================] - 0s 31ms/step
Model 1
Train score: 271.28
Test score: 166.06
MAPE Train:  7.3801723
Akurasi Train:  92.61982774734497
MAPE Test:  4.515449
Akurasi Test:  95.48455095291138


### Model 14 - 5

In [14]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(12, input_dim=seq_size,
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(10, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1)),
    tf.keras.layers.Dense(1, activation='sigmoid',
                          kernel_initializer=RandomUniform(minval=-1, maxval=1), bias_initializer=RandomUniform(minval=-1, maxval=1))
])

model1.compile(loss='mse',
               optimizer=tf.keras.optimizers.Adam(0.01),
               metrics=[tf.keras.metrics.MeanAbsoluteError()])

model1.fit(trainX, trainY, validation_data=(
    testX, testY), verbose=0, epochs=50, batch_size=6)

model1.evaluate(testX, testY)

trainPredict1 = model1.predict(trainX)
testPredict1 = model1.predict(testX)

trainPredict1 = scaler.inverse_transform(trainPredict1)
trainY_inverse = scaler.inverse_transform([trainY])
testPredict1 = scaler.inverse_transform(testPredict1)
testY_inverse = scaler.inverse_transform([testY])

print('Model 1')

trainScore1 = math.sqrt(mse(trainY_inverse[0], trainPredict1[:,0]))
print('Train score: %.2f' % (trainScore1))

testScore1 = math.sqrt(mse(testY_inverse[0], testPredict1[:,0]))
print('Test score: %.2f' % (testScore1))

mapeTrain1 = mape(trainY_inverse, trainPredict1)
print('MAPE Train: ', mapeTrain1.numpy())
print('Akurasi Train: ', 100 - (mapeTrain1.numpy()))

mape1 = mape(testY_inverse, testPredict1)
print('MAPE Test: ', mape1.numpy())
print('Akurasi Test: ', 100 - (mape1.numpy()))

1/1 [==============================] - 0s 33ms/step
Model 1
Train score: 281.30
Test score: 172.54
MAPE Train:  7.455037
Akurasi Train:  92.5449628829956
MAPE Test:  4.3185863
Akurasi Test:  95.6814136505127
